<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [29]:
# %load "load_libraries_and_datasets.py"
## 1. Load libraries and set env 
import warnings
warnings.filterwarnings('ignore')

## data structures and manipulation 
import pandas as pd
import numpy as np 

## visualization 
import seaborn as sns
import matplotlib.pyplot as plt 

## particular datatypes manipulation 
from dateutil.parser import parse

## statistical analysis - Ho-testing and glm Regression 
import statsmodels.api as sm
import statsmodels.formula.api as smf
## patsy is a Python library for describing statistical models and building Design Matrices using R-like formulas.
import patsy 

import scipy.stats as sci 
import math 


## Shared Env
%matplotlib inline 
plt.style.use('seaborn-white')
sns.set(color_codes=True)


## allow multiple outputs per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Getting R in the mix =D  << TODO: put R libs in a seperate Rscript file and load it from here<<< Excited!!!!!
# %load_ext rpy2.ipython 
# %R require(ggplot2); require(tidyr); require(dplyr) 

In [2]:
# help(patsy)
## Functions 
def sampleStructure(dtset, grp):
    argz = { 
        "CHP ID": {"# CHPs": "count"},
        "Households Registered": { "# HHs": "sum", "Avg.CHP.Size": "mean"},
        "Pregnancy" : { "# Pregoz Cases": "sum"},
        "All Malaria Assessments" : { "# Malaria Cases": "sum"},
        "Total U5 Assessments" : { "# Child Cases": "sum"},
        "Total U1 Assessments" : { "# U1 Cases": "sum"}
    }
    return dtset.groupby(grp).agg( argz).round(0)


## TODO: with margins + fq% 
def summarizer(dtset): 
    return dtset.describe().round(2)

## TODO: update to sns and with group by
def distributionz(dtset, colz, grpz=""):
    fig, ax = plt.subplots( figsize=(10,10))
    # return dtset.plot( colz, kind="hist", legend=True, alpha=0.4, ax=ax ) 
    return dtset.hist(colz,stacked=True,ax=ax)

def boxplotz(dtset, colz, grp):
    fig, ax = plt.subplots( figsize=(10,10))
    return dtset.boxplot( colz, grp, ax)

def corrz(dtset, colz):
    return dtset[colz].corr().round(3)
    
def plotCorrz( cor_outz ):
    # Generate a mask for the upper triangle
    mask = np.zeros_like(cor_outz, dtype=np.bool)
    mask[np.triu_indices_from(mask)] = True

    # Set up the matplotlib figure
    f, ax = plt.subplots(figsize=(11, 9))

    # Generate a custom diverging colormap
    cmap = sns.diverging_palette(220, 10, as_cmap=True)

    # Draw the heatmap with the mask and correct aspect ratio
    return cor_outz, sns.heatmap(cor_outz, mask=mask, cmap=cmap, vmax=.3, center=0,
                square=True, linewidths=.5, cbar_kws={"shrink": .5})

## TODO: generalize to smalls 
def quintilize(dtset, col, nbGrpz=5, labelzPrefix="Quint"):
    labelz = []
    for i in range(nbGrpz):
        labelz.append( labelzPrefix+"_"+str(i+1))
    dtset["Quints_"+col] = pd.qcut( dtset[col], nbGrpz, labels=labelz, duplicates='drop') 
    

def checkMissing(dtset):
    #dataset.fillna( np.nan )
    return "\nShape # records, # variables", dtset.shape, "\n\n\n Counts of Missing Values \n", dtset.apply( lambda x: sum(x.isnull() ), axis=0)






In [1]:
%quickref

In [2]:
!pip list

alabaster (0.7.10)
anaconda-client (1.6.5)
anaconda-navigator (1.6.9)
anaconda-project (0.8.0)
AnyQt (0.0.8)
asn1crypto (0.22.0)
astroid (1.5.3)
astropy (2.0.2)
babel (2.5.0)
backports.shutil-get-terminal-size (1.0.0)
beautifulsoup4 (4.6.0)
bitarray (0.8.1)
bkcharts (0.2)
blaze (0.11.3)
bleach (2.0.0)
bokeh (0.12.7)
boto (2.48.0)
Bottleneck (1.2.1)
CacheControl (0.12.3)
certifi (2017.7.27.1)
cffi (1.10.0)
chardet (3.0.4)
click (6.7)
cloudpickle (0.4.0)
clyent (1.2.2)
colorama (0.3.9)
comtypes (1.1.2)
conda (4.3.34)
conda-build (3.0.22)
conda-verify (2.0.0)
contextlib2 (0.5.5)
cryptography (2.0.3)
cycler (0.10.0)
Cython (0.26.1)
cytoolz (0.8.2)
dask (0.15.2)
datashape (0.5.4)
decorator (4.1.2)
dill (0.2.6)
distlib (0.2.5)
distributed (1.18.3)
docutils (0.14)
entrypoints (0.2.3)
et-xmlfile (1.0.1)
fastcache (1.0.2)
filelock (2.0.12)
Flask (0.12.2)
Flask-Cors (3.0.3)
gevent (1.2.2)
glob2 (0.5)
glue-core (0.10.4)
glue-vispy-viewers (0.7.2)
glueviz (0.10.4)
greenlet (0.4.12)
h5py (2.7.0)
he

DEPRECATION: The default format will switch to columns in the future. You can use --format=(legacy|columns) (or define a format=(legacy|columns) in your pip.conf under the [list] section) to disable this warning.
